In [23]:
# -*- coding: utf-8 -*-
"""
Swarm density (POD/ACC) 日別グリッド化スクリプト（Python版）
- 依存: cdflib, numpy, scipy
    pip install cdflib scipy numpy
- 主な仕様:
    * CDF の変数名: time, density, altitude, latitude, longitude, local_solar_time
    * 時刻は TT2000/epoch を自動処理（cdflib.cdfepoch.to_datetime）
    * 30秒（POD）/10秒（ACC）ごとに 1 日を等間隔グリッドに詰める
    * 日番号（doy）で辞書 DNS[doy] に格納
    * MATLABの保存形式に近づけるため .mat も保存（Pythonからの読み出しも容易な .npz も同時保存）
"""

from pathlib import Path
from datetime import datetime, timezone
import numpy as np
from cdflib import CDF, cdfepoch
from scipy.io import savemat

def _to_datetimes(time_var):
    """cdflib の time 変数を Python datetime配列に変換（TT2000/epoch双方を許容）"""
    try:
        dt = np.array(cdfepoch.to_datetime(time_var))
        # to_datetime が返すのは list[datetime]; numpy配列にする
        return dt
    except Exception:
        # フォールバック: 秒 since UNIX epoch と仮定
        return np.array([datetime.fromtimestamp(float(t), tz=timezone.utc) for t in time_var])

def _sec_of_day(dt_array):
    """datetime配列 -> 秒/日の整数配列"""
    return (dt_array.astype('datetime64[s]') - dt_array.astype('datetime64[D]')).astype(int)

def _ymd_hms(dt_array):
    """datetime配列 -> (N,6) の [Y,M,D,h,m,s] int 配列"""
    out = np.empty((len(dt_array), 6), dtype=int)
    for i, d in enumerate(dt_array):
        out[i] = [d.year, d.month, d.day, d.hour, d.minute, d.second]
    return out

def _days_since_year_start(dt_array, year):
    """datetime配列 -> 年初からの経過日（小数日）"""
    first = np.datetime64(datetime(year, 1, 1, tzinfo=timezone.utc))
    return (dt_array.astype('datetime64[ns]') - first).astype('timedelta64[ns]').astype(np.float64) / (24*3600*1e9)

def _process_one_file(cdf_path, step_sec, expected_len, year):
    """
    1 つの CDF を読み、指定ステップ（秒）で 1 日の等間隔グリッドに詰める。
    返り値: dict(fields) と doy(int)
    """
    with CDF(str(cdf_path)) as cdf:
        den = np.array(cdf.varget('density'), dtype=float).ravel()
        alt = np.array(cdf.varget('altitude'), dtype=float).ravel()
        lat = np.array(cdf.varget('latitude'), dtype=float).ravel()
        lon = np.array(cdf.varget('longitude'), dtype=float).ravel()
        lts = np.array(cdf.varget('local_solar_time'), dtype=float).ravel()
        t_raw = cdf.varget('time')

    dts = _to_datetimes(t_raw)
    # 念のためUTCに正規化（tz情報なしならUTCとみなす）
    dts = np.array([d if d.tzinfo else d.replace(tzinfo=timezone.utc) for d in dts])
    ymd = _ymd_hms(dts)
    sod = _sec_of_day(dts.astype('datetime64[ns]'))  # 秒/日（0〜86399）
    doy = int(dts[-1].timetuple().tm_yday)  # その日のDOYを代表として採用
    t_days = _days_since_year_start(dts.astype('datetime64[ns]'), year)  # 年初からの小数日

    # 出力器
    out = {
        'Den': np.full(expected_len, np.nan, dtype=float),
        'alt': np.full(expected_len, np.nan, dtype=float),
        'lat': np.full(expected_len, np.nan, dtype=float),
        'lon': np.full(expected_len, np.nan, dtype=float),
        'LT':  np.full(expected_len, np.nan, dtype=float),
        'T':   np.full(expected_len, np.nan, dtype=float),   # 年初からの小数日（MATLAB版の T-first_day に対応）
        'ymd': np.full((expected_len, 6), np.nan, dtype=float),
    }

    # もし既に等間隔ならそのまま詰める
    if len(den) == expected_len:
        out['Den'][:] = den
        out['alt'][:] = alt
        out['lat'][:] = lat
        out['lon'][:] = lon
        out['LT'] [:] = lts
        out['T']  [:] = t_days
        out['ymd'][:] = ymd
        return out, doy

    # 等間隔でない場合はビン詰め
    # インデックス = floor(秒/日 ÷ step_sec)
    idx = (sod // step_sec).astype(int)
    # 1 日の範囲外があれば除去（基本的に出ない想定）
    mask = (idx >= 0) & (idx < expected_len)
    idx = idx[mask]
    if idx.size:
        out['Den'][idx] = den[mask]
        out['alt'][idx] = alt[mask]
        out['lat'][idx] = lat[mask]
        out['lon'][idx] = lon[mask]
        out['LT'][idx]  = lts[mask]
        out['T'][idx]   = t_days[mask]
        out['ymd'][idx] = ymd[mask]

    return out, doy

def process_year(choice='POD',
                    year=2020,
                    base_dir=Path(r'data/swarm/2020_POD'),
                    sat='Sat_A',
                    subfolder='new',
                    save_mat=True,
                    save_npz=True):
    """
    choice : 'POD' or 'ACC'
    year   : 対象年
    base_dir/sat/year/subfolder 配下の CDF を処理
    保存先は base_dir/<choice>/<sat>/mat/<year>/<subfolder>/DNS.mat （と DNS.npz）
    """
    choice = choice.upper()
    if choice not in ('POD', 'ACC'):
        raise ValueError("choice must be 'POD' or 'ACC'")

    if choice == 'POD':
        step_sec = 30
        expected_len = 2880
        in_dir = base_dir / 'POD' / sat / f'{year}' / subfolder
        out_dir = base_dir / 'POD' / sat / 'mat' / f'{year}' / subfolder
    else:
        step_sec = 10
        expected_len = 8640
        # 元MATLABは Sat_C を使っているのでデフォルト C に
        if sat is None:
            sat = 'Sat_C'
        in_dir = base_dir / 'ACC' / sat / f'{year}'
        out_dir = base_dir / 'ACC' / sat / 'mat' / f'{year}'

    out_dir.mkdir(parents=True, exist_ok=True)

    # CDF列挙
    cdf_list = sorted([p for p in in_dir.glob('*.cdf')] + [p for p in in_dir.glob('*.CDF')])
    if not cdf_list:
        raise FileNotFoundError(f'No CDF files found in: {in_dir}')

    DNS = {}  # doy(int) -> dict(fields)

    for cdf_path in cdf_list:
        rec, doy = _process_one_file(cdf_path, step_sec, expected_len, year)
        # もし同一DOYが複数ファイルに分かれている場合、後勝ち or 空いているところだけ上書き
        if doy not in DNS:
            DNS[doy] = rec
        else:
            # 既存の NaN を新データで埋める
            for k in ('Den', 'alt', 'lat', 'lon', 'LT', 'T'):
                m = np.isnan(DNS[doy][k]) & ~np.isnan(rec[k])
                DNS[doy][k][m] = rec[k][m]
            m2 = np.all(np.isnan(DNS[doy]['ymd']), axis=1) & ~np.all(np.isnan(rec['ymd']), axis=1)
            DNS[doy]['ymd'][m2] = rec['ymd'][m2]

    # 保存（MATLAB互換とPython向け両方）
    # savemat はネスト辞書をそこそこ扱えます（MATLAB側では struct/containers.Map として参照可能）
    if save_mat:
        mat_path = out_dir / 'DNS.mat'
        # MATLAB で扱いやすいように、文字列キーを 'doy_<NNN>' にしておく
        dns_for_mat = {f'doy_{k:03d}': v for k, v in DNS.items()}
        savemat(str(mat_path), {'DNS': dns_for_mat})
        print(f'Saved: {mat_path}')

    if save_npz:
        npz_path = out_dir / 'DNS.npz'
        # npz はネスト構造をそのまま保存できないため、各DOYを個別に保存
        # 例: Den_001, alt_001, ..., Den_365
        pack = {}
        for k, rec in DNS.items():
            tag = f'{k:03d}'
            for field, arr in rec.items():
                pack[f'{field}_{tag}'] = arr
        np.savez_compressed(str(npz_path), **pack)
        print(f'Saved: {npz_path}')

    return DNS

if __name__ == '__main__':
    # === 使い方例 ===
    # POD (30s) の 2020 年、Windowsパス構成を模したデフォルトで処理
    DNS_pod = process_year(choice='POD',
                            year=2020,
                            base_dir=Path(r'data/swarm/2020_POD'),
                            sat='Sat_A',
                            subfolder='new',
                            save_mat=True,
                            save_npz=True)

    # ACC (10s) の 2024 年、Sat_C を処理したい場合は以下をコメントアウト解除
    # DNS_acc = process_year(choice='ACC',
    #                        year=2024,
    #                        base_dir=Path(r'E:\data\Swarm\DNS'),
    #                        sat='Sat_C',
    #                        subfolder='',
    #                        save_mat=True,
    #                        save_npz=True)


FileNotFoundError: No CDF files found in: data/swarm/2020_POD/POD/Sat_A/2020/new